## Load frames

In [7]:
import os
import pickle
import re
import subprocess
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf
import tensorflow_datasets as tfds
from IPython import display
from matplotlib import font_manager as fm
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import time
import base64
import io
import imageio
import json
import os
from typing import Dict, Any

2025-08-18 17:02:43.808928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755507763.822805  644789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755507763.827054  644789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755507763.839861  644789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755507763.839875  644789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755507763.839876  644789 computation_placer.cc:177] computation placer alr

In [8]:
def dataset2path(dataset_name):
    
    try:
        # 调用 gsutil ls 命令获取所有版本
        cmd = f'gsutil ls gs://gresearch/robotics/{dataset_name}/'
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        
        if result.returncode != 0:
            print(f"警告: 无法访问 gs://gresearch/robotics/{dataset_name}/，错误信息: {result.stderr}")
            # 如果失败，使用默认版本号逻辑
            if dataset_name == 'robo_net':
                default_version = '1.0.0'
            elif dataset_name in ['language_table', 'robo_set', 'spoc',"DROID"]:
                default_version = '0.0.1'
            elif dataset_name in ['droid']:
                default_version = '1.0.0'
            else:
                default_version = '0.1.0'
            print(f"使用默认版本号: {default_version}")
            return f'gs://gresearch/robotics/{dataset_name}/{default_version}'
        
        # 解析输出，查找版本号格式 数字.数字.数字 或 数字.数字
        lines = result.stdout.strip().split('\n')
        versions = []
        
        for line in lines:
            if line.strip():
                # 提取路径中的版本号部分
                # 格式: gs://gresearch/robotics/{dataset_name}/{version}/
                match = re.search(rf'gs://gresearch/robotics/{re.escape(dataset_name)}/(\d+\.\d+(?:\.\d+)?)', line)
                if match:
                    version_str = match.group(1)
                    versions.append(version_str)
        
        if not versions:
            print(f"警告: 未找到有效的版本号，使用默认版本")
            # 使用默认版本号逻辑
            if dataset_name == 'robo_net':
                backup_version = '1.0.0'
            elif dataset_name in ['language_table', 'robo_set', 'spoc',"DROID"]:
                backup_version = '0.0.1'
            elif dataset_name in ['droid']:
                backup_version = '1.0.0'
            else:
                backup_version = '0.1.0'
            print(f"使用默认版本号: {backup_version}")
            return f'gs://gresearch/robotics/{dataset_name}/{backup_version}'
        
        # 对版本号进行排序，找到最小版本
        def version_key(version_str):
            try:
                parts = version_str.split('.')
                return tuple(int(part) for part in parts)
            except ValueError:
                # 如果版本号格式有问题，返回一个很大的数字作为排序键
                return (999, 999, 999)
        
        # 去重版本号列表
        unique_versions = list(set(versions))
        min_version = max(unique_versions, key=version_key)
        print(f"数据集 {dataset_name} 找到版本: {versions}，去重后: {unique_versions}，选择最小版本: {min_version}")
        print(f'正常返回 gs://gresearch/robotics/{dataset_name}/{min_version}')
        return f'gs://gresearch/robotics/{dataset_name}/{min_version}'
        
    except Exception as e:
        print(f"错误: 获取数据集 {dataset_name} 版本时出错: {e}")
        # 出错时使用默认版本号逻辑
        if dataset_name == 'robo_net':
            fallback_version = '1.0.0'
        elif dataset_name in ['language_table', 'robo_set', 'spoc',"DROID"]:
            fallback_version = '0.0.1'
        elif dataset_name in ['droid']:
            fallback_version = '1.0.0'
        else:
            fallback_version = '0.1.0'
        print(f"使用默认版本号: {fallback_version}")
        print(f'异常 gs://gresearch/robotics/{dataset_name}/{min_version}')
        return f'gs://gresearch/robotics/{dataset_name}/{fallback_version}'

def as_gif(images, path='temp.gif', duration=1000):
  # Render the images as the gif:
  images[0].save(path, save_all=True, append_images=images[1:], duration=duration, loop=0)
  gif_bytes = open(path,'rb').read()
  return gif_bytes


def save_images_to_video_with_imageio(image_list, output_path, fps=30):
    """
    使用imageio将PIL.Image列表保存为视频
    
    参数:
        image_list: PIL.Image对象的列表
        output_path: 输出视频文件路径
        fps: 帧率
    """
    if not image_list:
        print("错误: 图像列表为空")
        return
    
    # 转换为numpy数组列表
    frames = [np.array(img) for img in image_list]
    
    # 保存为视频
    imageio.mimsave(output_path, frames, fps=fps)
    print(f"视频已保存到 {output_path}")

def images_to_base64(image_list):
    base64Frames = []
    for img in image_list:
        if isinstance(img, np.ndarray):
            img = Image.fromarray(img)
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='JPEG')
        img_byte_arr = img_byte_arr.getvalue()
        base64Frames.append(base64.b64encode(img_byte_arr).decode("utf-8"))
    frame_count = len(base64Frames)
    print(f"{frame_count} frames processed.")
    return base64Frames, frame_count

In [9]:

class TaskVidCaptionManager:
    def __init__(self, file_path: str):
        """
        初始化管理器
        
        Args:
            file_path (str): JSON 文件路径
        """
        self.file_path = file_path
        self.data = self._load_or_initialize()
    
    def _load_or_initialize(self) -> Dict[str, Any]:
        """
        加载现有文件或初始化空数据结构
        
        Returns:
            Dict: 加载的数据或新初始化的空数据结构
        """
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r', encoding='utf-8') as f:
                try:
                    return json.load(f)
                except json.JSONDecodeError:
                    print(f"Warning: {self.file_path} is corrupted, initializing new data")
                    return self._initialize_empty_data()
        else:
            return self._initialize_empty_data()
    
    def _initialize_empty_data(self) -> Dict[str, Any]:
        """
        初始化空的数据结构
        
        Returns:
            Dict: 空的三层嵌套结构
        """
        return {
            # 结构: {task_name: {vid: caption}}
        }
    
    def save(self):
        """
        将当前数据保存到文件
        """
        with open(self.file_path, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, indent=4, ensure_ascii=False)
    
    def add_entry(self, task_name: str, vid: str, caption: str):
        """
        添加新的条目
        
        Args:
            task_name (str): 任务名称
            vid (str): 视频ID
            caption (str): 视频描述
        """
        if task_name not in self.data:
            self.data[task_name] = {}
        
        self.data[task_name][vid] = caption
        self.save()
    
    def get_caption(self, task_name: str, vid: str) -> str:
        """
        获取特定任务和视频的caption
        
        Args:
            task_name (str): 任务名称
            vid (str): 视频ID
            
        Returns:
            str: 视频描述，如果不存在则返回None
        """
        return self.data.get(task_name, {}).get(vid)
    
    def get_task_vids(self, task_name: str) -> Dict[str, str]:
        """
        获取特定任务的所有vid和caption映射
        
        Args:
            task_name (str): 任务名称
            
        Returns:
            Dict: {vid: caption} 的映射，如果任务不存在则返回空字典
        """
        return self.data.get(task_name, {})
    
    def remove_entry(self, task_name: str, vid: str):
        """
        删除特定条目
        
        Args:
            task_name (str): 任务名称
            vid (str): 视频ID
        """
        if task_name in self.data and vid in self.data[task_name]:
            del self.data[task_name][vid]
            # 如果任务下没有视频了，也删除任务
            if not self.data[task_name]:
                del self.data[task_name]
            self.save()

## 下载最新的 google sheet

In [13]:
sheet_id = "13Q7OtTh50BKiHMxa_t0g0Yz44VcEeRlhvlmSOoGUPGw"
gid = "1221129144"
!curl -L "https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}" -o /home2/qrchen/embodied-datasets/scripts/xintong/Datasets/metadata.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431    0   431    0     0   1159      0 --:--:-- --:--:-- --:--:--  1158
100  188k    0  188k    0     0   196k      0 --:--:-- --:--:-- --:--:--  196k


## Choose dataset

修改要处理的数据集的配置：
- 数据集名称
- image 字段名称

缓存 episode 的列表，方便后续处理

In [ ]:
# 数据集配置
DATASET_NAME = "BridgeData"        # 数据集名称
MAX_EPISODES = 1
df = pd.read_csv('/home2/qrchen/embodied-datasets/scripts/xintong/Datasets/metadata.csv') # 可以修改为其他路径

# 从df 中找到 DATASET_CONFIG 中 dataset_name_in_csv 对应的行，得到其中 nickname 的值
dataset_name_in_csv = DATASET_NAME
dataset = df[df['Datasets'] == dataset_name_in_csv]
dataset = dataset['NickName'].item()

# 设置保存路径
save_root = os.path.join('/home2/qrchen/embodied-datasets/Modifications', dataset)
os.makedirs(save_root, exist_ok=True)



# 加载数据集并查看其 task 和 episode 数量
b = tfds.builder_from_directory(builder_dir=dataset2path(dataset))
ds = b.as_dataset(split='train').take(MAX_EPISODES)  # 限制最大 episode 数量
print(b.info.features)
print(f"Dataset: {dataset}, Total Episodes: {b.info.splits['train'].num_examples}")

# 配置
MAX_TASKS = 10
EPISODES_PER_TASK = 10
STRICT_MODE = True  # 是否严格模式
# STRICT_MODE = True  -> 直到收齐 10 个 task 且每个 task 都 >= 10 个 episode 才停
# STRICT_MODE = False -> 一旦遇到 10 个不同的 task 就停（不保证每个 task 都有 10 个 episode）

task2count = defaultdict(int)
episode2instruction = {}                 # episode_id -> instruction
instruction2episodes = defaultdict(list) # instruction -> [episode_id, ...]
unique_tasks = set()  # 不同的task 的集合，会去除重复的task

def extract_instruction_from_step0(step_0):
    """根据字段结构提取 instruction 文本。"""
    obs = step_0['observation']
    if 'natural_language_instruction' in obs:
        return obs['natural_language_instruction'].numpy().decode('utf-8')
    if 'language_instruction' in step_0:
        return step_0['language_instruction'].numpy().decode('utf-8')
    instruction_bytes = obs['instruction']
    return instruction_bytes.numpy().decode('utf-8').split('\x00')[0]

def done():
    """是否满足停止条件。"""
    if len(unique_tasks) < MAX_TASKS:
        return False
    if not STRICT_MODE:
        return True
    # 严格模式：10 个 task 且每个 task 都 >= 10 个 episode
    for t in unique_tasks:
        if len(instruction2episodes[t]) < EPISODES_PER_TASK:
            return False
    return True

episodes = []
episode_id = 0
pbar = tqdm(ds, desc="统计 task 构建索引并缓存 episode")
for episode in pbar:
    try:
        if episode_id >= MAX_EPISODES:
            print(f"已达到最大episode数 {MAX_EPISODES}，提前终止。")
            break
        episodes.append(episode)  # 缓存 episode
        step_0 = next(iter(episode['steps']))  # 仅取第 0 步
        instruction = extract_instruction_from_step0(step_0).replace('\n','')
        # 统计与索引
        task2count[instruction] += 1
        episode2instruction[episode_id] = instruction
        instruction2episodes[instruction].append(episode_id)
        prev_task_count = len(unique_tasks)
        unique_tasks.add(instruction)
        # 每有新task就刷新进度条显示
        if len(unique_tasks) != prev_task_count:
            pbar.set_postfix({"unique_tasks": len(unique_tasks)})
        if done():
            break
    except Exception as e:
        print(f"[跳过] episode {episode_id} 出错：{e}")
    finally:
        episode_id += 1

print(f"已采集到 {len(unique_tasks)} 个不同 task")
print(f"unique_tasks: {list(unique_tasks)}")

os.makedirs(save_root, exist_ok=True)
if STRICT_MODE:
    print("各 task 收集 episode 数量(上限 10): ")
    for t in list(unique_tasks)[:MAX_TASKS]:
        print(f"  {t[:60]}... -> {len(instruction2episodes[t])}")

# —— 保存结果，便于后续随机访问/分析 ——
df_map = pd.DataFrame(
    {"episode_id": list(episode2instruction.keys()),
     "instruction": [episode2instruction[eid] for eid in episode2instruction]}
)
df_map.to_csv(os.path.join(save_root, "episode_instruction_map.csv"), index=False)

# 保存 每个指令对应的 episode_id 列表
with open(os.path.join(save_root, "instruction2episodes.json"), "w", encoding="utf-8") as f:
    json.dump(instruction2episodes, f, indent=2, ensure_ascii=False)


数据集 bridge 找到版本: ['0.1.0', '0.1.0']，去重后: ['0.1.0']，选择最小版本: 0.1.0
正常返回 gs://gresearch/robotics/bridge/0.1.0


W0000 00:00:1755507782.552955  644789 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


FeaturesDict({
    'steps': Dataset({
        'action': FeaturesDict({
            'open_gripper': bool,
            'rotation_delta': Tensor(shape=(3,), dtype=float32),
            'terminate_episode': float32,
            'world_vector': Tensor(shape=(3,), dtype=float32),
        }),
        'is_first': bool,
        'is_last': bool,
        'is_terminal': bool,
        'observation': FeaturesDict({
            'image': Image(shape=(480, 640, 3), dtype=uint8),
            'natural_language_embedding': Tensor(shape=(512,), dtype=float32),
            'natural_language_instruction': string,
            'state': Tensor(shape=(7,), dtype=float32),
        }),
        'reward': Scalar(shape=(), dtype=float32),
    }),
})
Dataset: bridge, Total Episodes: 25460


统计 task 构建索引并缓存 episode:   0%|          | 0/1 [00:00<?, ?it/s]2025-08-18 17:03:02.706398: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


## 从 episode 中提取 video 并保存

默认一次转换并保存10个video，如果这个task的 episode 不足10个，则处理全部。
若想处理这个 task 的其他 episode，可以修改 `start_idx`

- **注意修改 image 字段名称**，如果是 `image` 则不需要修改

In [ ]:
import os
import imageio
from tqdm import tqdm
import numpy as np

IMAGE = 'image_0'  # 修改为实际的图像字段名称，如果是 'image' 则不需要修改
TASK_INDEX = 0  # 你要可视化的 task 的索引
START_IDX = 0  # 从所选 task 第几个 episode 开始
ALL_TASKS = True # 是否可视化所有的task
fps = 20


# 得到annotation 类
annotation_file = os.path.join(save_root, 'annotations.json')
manager = TaskVidCaptionManager(annotation_file) #定义了一个类，task中 caption 标注的对应关系
def extract_frames_from_episode(episode, max_frames=None, frame_stride=1):
    steps = list(episode['steps'])
    frames = []
    for i, step in enumerate(steps[::frame_stride]):
        img = step['observation'][IMAGE]
        img = img.numpy()
        if img.ndim == 3 and img.shape[-1] in (1, 3, 4):
            pass
        else:
            if img.ndim == 2:
                img = np.stack([img]*3, axis=-1)
            elif img.ndim == 3 and img.shape[0] in (1,3,4):
                img = np.transpose(img, (1,2,0))
        frames.append(img)
        if (max_frames is not None) and (len(frames) >= max_frames):
            break
    return frames

def get_episode_by_id(episodes, eid):
    # 直接索引
    return episodes[eid]

def visualize_task_episodes(episodes, instruction2episodes, task, start_idx=0, count=10, save_root='.', fps=3, max_frames=None, frame_stride=1):
    os.makedirs(save_root, exist_ok=True)
    episode_ids = instruction2episodes[task]
    selected_ids = episode_ids[start_idx:start_idx+count]
    if len(selected_ids) < count:
        print(f"警告: 只找到了 {len(selected_ids)} 个 episode，少于期望的 {count} 个。")
    for eid in tqdm(selected_ids, desc=f"保存 {task[:20]}"):
        # 把对应的eid 对应 “TODO” 的caption 保存到manager中
        manager.add_entry(task,f'{eid}',"TODO caption")
        episode = get_episode_by_id(episodes, eid)
        frames = extract_frames_from_episode(episode, max_frames=max_frames, frame_stride=frame_stride)
        frames_uint8 = [(f if f.dtype == np.uint8 else (f * 255).astype(np.uint8)) for f in frames]
        video_path = os.path.join(save_root, f"{task[:30].replace(' ','_').replace('/','_')}_ep{eid}.mp4")
        imageio.mimsave(video_path, frames_uint8, fps=fps)
        print(f"已保存: {video_path}")

# 用法示例
task = list(instruction2episodes.keys())[TASK_INDEX]  # 你要可视化的第TASK_NUM个task
all_tasks = list(instruction2episodes.keys()) # 可视化所有的task
start_idx = START_IDX  # 从第几个episode开始 可以查看 instruction2episodes 文件来修改
#如果可视化all task，则可视化所有的task，否则只可视化指定的task
if ALL_TASKS:
    for task in all_tasks:
        save_dir = os.path.join(save_root, 'samples', task[:30].replace(' ','_').replace('/','_').replace('\n','')) #避免过长的文件名
        visualize_task_episodes(episodes, instruction2episodes, task, start_idx=start_idx, count=10, save_root=save_dir, fps=fps)
else:
    save_dir = os.path.join(save_root, 'samples', task[:30].replace(' ','_').replace('/','_')) #避免过长的文件名
    visualize_task_episodes(episodes, instruction2episodes, task, start_idx=start_idx, count=10, save_root=save_dir, fps=fps)
manager.save()

## 可视化选取的 task 的 episode

默认一次可视化10个，如果这个task的 episode 不足10个，则可视化全部。
若想可视化这个 task 的其他 episode，可以修改 `start_idx`

In [ ]:
from IPython.display import display, HTML
import glob
import os

# 构造本次要展示的视频文件名列表
count = 10  # 每次展示的视频数量
episode_ids = instruction2episodes[task]
selected_ids = episode_ids[start_idx:start_idx+count]
video_files = [
    os.path.join(save_dir, f"{task[:30].replace(' ','_').replace('/','_')}_ep{eid}.mp4")
    for eid in selected_ids
    if os.path.exists(os.path.join(save_dir, f"{task[:30].replace(' ','_').replace('/','_')}_ep{eid}.mp4"))
]

html = "<div style='display:flex;flex-direction:column;'>"
for i in range(0, len(video_files), 5):
    html += "<div style='display:flex;flex-direction:row;'>"
    for video_path in video_files[i:i+5]:
        html += f"<video src='{video_path}' width='180' height='135' controls loop autoplay muted style='margin:2px;'></video>"
    html += "</div>"
html += "</div>"

display(HTML(html))

## 可视化 modification

In [1]:
import os
import json

def generate_modifications_index(dataset_dir, videos_per_row=4):
    import os, json

    samples_dir = os.path.join(dataset_dir, 'samples')
    annotations_path = os.path.join(dataset_dir, 'annotations.json')
    if not (os.path.exists(samples_dir) and os.path.exists(annotations_path)):
        print(f"跳过 {dataset_dir}，缺少 samples 或 annotations.json")
        return

    with open(annotations_path, 'r', encoding='utf-8') as f:
        annotations = json.load(f)

    html = [
        '<!DOCTYPE html>',
        '<html lang="zh-cn">',
        '<head>',
        '<meta charset="UTF-8">',
        '<title>Modifications Dataset Viewer</title>',
        '<style>',
        "body { font-family: 'Segoe UI', Arial, sans-serif; background: #f7f7fa; padding: 30px; }",
        '.tasks-row { display: flex; flex-direction: column; gap: 48px; margin-bottom: 48px; }',
        '.task-block { background: #fff; border-radius: 14px; box-shadow: 0 2px 12px #e0e0f6; padding: 28px; width: 100%; }',
        '.instruction-origin { font-size: 1.15rem; color: #764ba2; font-weight: 600; margin-bottom: 20px; }',
        '.instruction-modified { font-size: 1rem; color: #27ae60; font-weight: 500; }',
        '.video-list { display: grid; grid-template-columns: repeat(VIDEO_COLS, 1fr); gap: 20px; margin-top: 10px; }'.replace('VIDEO_COLS', str(videos_per_row)),
        '.video-item { background: #f8faff; border-radius: 8px; box-shadow: 0 1px 6px #eee; padding: 8px; }',
        '.caption { font-size: 0.95rem; color: #666; margin-top: 4px; }',
        '.back-btn { display:inline-block; margin-bottom:18px; padding:8px 22px; background:#4e6ef2; color:#fff; border-radius:6px; text-decoration:none; font-size:1rem; font-weight:500; transition:background 0.2s; }',
        '.back-btn:hover { background:#2d4ecf; }',
        '.top-btn { position:fixed; right:36px; bottom:36px; z-index:99; background:#764ba2; color:#fff; border:none; border-radius:8px; padding:12px 22px; font-size:1.1rem; font-weight:600; cursor:pointer; box-shadow:0 2px 8px #aaa; transition:background 0.2s; }',
        '.top-btn:hover { background:#4e6ef2; }',
        '</style>',
        '</head>',
        '<body>',
        '<a class="back-btn" href="../../index.html">&larr; Back</a>',
        f'<h1>{os.path.basename(dataset_dir)} Dataset Viewer</h1>',
        '<button class="top-btn" onclick="window.scrollTo({top:0,behavior:\'smooth\'});">返回顶部</button>'
    ]

    tasks = list(annotations.items())
    for task, vids in tasks:
        slug = task[:30].replace(' ', '_').replace('/', '_')
        task_dir = os.path.join(samples_dir, slug)
        if not os.path.isdir(task_dir):
            continue

        # 只处理有标注的视频
        if not vids:  
            continue

        html.append('<div class="task-block">')
        html.append(f'<div class="instruction-origin">ORIGIN: {task.replace("<", "&lt;").replace(">", "&gt;")}</div>')

        video_files = [f for f in os.listdir(task_dir) if f.endswith('.mp4')]
        
        # 按 ep_id 数字大小排序视频文件
        def get_ep_id(fname):
            try:
                return int(fname.split('_ep')[-1].replace('.mp4', ''))
            except ValueError:
                return 0  # 如果解析失败，返回0
        
        video_files.sort(key=get_ep_id)
        
        html.append('<div class="video-list">')
        for fname in video_files:
            video_path = f'samples/{slug}/{fname}'
            ep_id = fname.split('_ep')[-1].replace('.mp4', '')
            caption = vids.get(ep_id, None)  # 获取标注，如果没有则为None
            
            # 只有存在标注时才显示这个视频
            if caption is not None:
                html.append('<div class="video-item">')
                html.append(f'<video src="{video_path}" controls width="100%" loop autoplay muted></video>')
                html.append(f'<div class="caption"><b>Video:</b> {fname}</div>')
                html.append(f'<div class="instruction-modified">MODIFY : {caption}</div>')
                html.append('</div>')
        
        html.append('</div>')  # .video-list
        html.append('</div>')  # .task-block

    with open(os.path.join(dataset_dir, 'index.html'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(html))
    print(f"已生成: {os.path.join(dataset_dir, 'index.html')}")

# 用法示例
modifications_root = "/home2/qrchen/embodied-datasets/Modifications"
for dataset_name in os.listdir(modifications_root):
    dataset_dir = os.path.join(modifications_root, dataset_name)
    if not os.path.isdir(dataset_dir):
        continue
    try:
        generate_modifications_index(dataset_dir, videos_per_row=4)
    except Exception as e:
        print(f"跳过 {dataset_dir}，原因: {e}")

跳过 /home2/qrchen/embodied-datasets/Modifications/nyu_door_opening_surprising_effectiveness，原因: [Errno 13] Permission denied: '/home2/qrchen/embodied-datasets/Modifications/nyu_door_opening_surprising_effectiveness/index.html'
跳过 /home2/qrchen/embodied-datasets/Modifications/cmu_franka_exploration_dataset_converted_externally_to_rlds，原因: [Errno 13] Permission denied: '/home2/qrchen/embodied-datasets/Modifications/cmu_franka_exploration_dataset_converted_externally_to_rlds/index.html'
跳过 /home2/qrchen/embodied-datasets/Modifications/ucsd_pick_and_place_dataset_converted_externally_to_rlds，原因: [Errno 13] Permission denied: '/home2/qrchen/embodied-datasets/Modifications/ucsd_pick_and_place_dataset_converted_externally_to_rlds/index.html'
跳过 /home2/qrchen/embodied-datasets/Modifications/viola，原因: [Errno 13] Permission denied: '/home2/qrchen/embodied-datasets/Modifications/viola/index.html'
已生成: /home2/qrchen/embodied-datasets/Modifications/berkeley_fanuc_manipulation/index.html
跳过 /home2/qr

## calculate the number of tasks 

In [5]:
import os
import json

total_difference = 0
DATASET_list=[
        "droid",
        "dobbe",
        "fmb",
        "aloha_mobile",
        "io_ai_tech",
        "robo_set",
        "uiuc_d3field",
        "utaustin_mutex",
        "berkeley_fanuc_manipulation",
        "cmu_playing_with_food",
        "cmu_play_fusion",
        "cmu_stretch",
        "mimicplay",
        "bridge",
        "kuka"
]

Dataset_annotation = [
    "kuka",
]
base_dir = "/home2/qrchen/embodied-datasets/Modifications"
for dataset in DATASET_list:
    dataset_dir =  os.path.join(base_dir, dataset)
    if not os.path.isdir(dataset_dir):
        print(f"not find : {dataset}")
        continue
    if not os.path.exists(os.path.join(dataset_dir, "difference.json")):
        print(f"not find difference.json : {dataset}")
        continue
    with open(os.path.join(dataset_dir, "difference.json"), "r") as f:
        difference = json.load(f)

    for task in difference.keys():
        for difference_type in difference[task].keys():
            total_difference += 1
            print(difference_type)
print("total difference: ", total_difference)




for dataset in Dataset_annotation:
    dataset_dir =  os.path.join(base_dir, dataset)
    if not os.path.exists(os.path.join(dataset_dir,"annotations.json")):
        print(f"not find annotations.json : {dataset}")
        continue
    with open(os.path.join(dataset_dir, "annotations.json"), "r") as f:
        annotations = json.load(f)
    print(f"[total task numbers]dataset: {dataset} have {len(annotations.keys())} tasks")


place different position
interesting task , make the cup upside down
interesting task ,open the door of the  washing machine
interesting task
different handle 
rotate direction
round handle or square handle , decide whether need to rotate firstly
 different level of the cabinet
different direction of the move
after what to do after grasping the handle
rotate direction of the handle
place in different position(about the pick and place task , pick what and place where)
different way to pick object 
detail information of place 
pick different part of object
 how to switch the button
the contact part of the button
 switch both button or not 
switch different direction
how to open the drawer
12 . how to open the differnet door
13. Different ways to open sliding doors
14. left or right of the handle
15. how to push the drawer
one stage or two stage ? 
one stage or two stage (all <Pick up> task have such problems)
not enough information add insert information
not find difference.json : aloha_

## 新 visualizer 的代码


In [8]:
import os
import json
import base64

def generate_modifications_index(dataset_dir, videos_per_row=4):
    import os, json

    samples_dir = os.path.join(dataset_dir, 'samples')
    annotations_path = os.path.join(dataset_dir, 'annotations.json')
    if not (os.path.exists(samples_dir) and os.path.exists(annotations_path)):
        print(f"跳过 {dataset_dir}，缺少 samples 或 annotations.json")
        return

    with open(annotations_path, 'r', encoding='utf-8') as f:
        annotations = json.load(f)

    html = [
        '<!DOCTYPE html>',
        '<html lang="zh-cn">',
        '<head>',
        '<meta charset="UTF-8">',
        '<title>Modifications Dataset Viewer</title>',
        '<style>',
        "body { font-family: 'Segoe UI', Arial, sans-serif; background: #f7f7fa; padding: 30px; }",
        '.tasks-row { display: flex; flex-direction: column; gap: 48px; margin-bottom: 48px; }',
        '.task-block { background: #fff; border-radius: 14px; box-shadow: 0 2px 12px #e0e0f6; padding: 28px; width: 100%; }',
                 '.instruction-origin { font-size: 1.15rem; color: #3b82f6; font-weight: 600; margin-bottom: 20px; padding-bottom: 10px; border-bottom: 2px solid #bae6fd; }',
         '.instruction-modified { font-size: 1rem; color: #5bb8a7; font-weight: 500; }',
         '.video-list { display: grid; grid-template-columns: repeat(VIDEO_COLS, 1fr); gap: 20px; margin-top: 10px; }'.replace('VIDEO_COLS', str(videos_per_row)),
         '.video-item { background: #f0f9ff; border-radius: 8px; box-shadow: 0 2px 8px rgba(96, 165, 250, 0.1); padding: 12px; border: 1px solid #bae6fd; }',
         '.caption { font-size: 0.95rem; color: #64748b; margin-top: 8px; }',
         '.back-btn { display:inline-block; margin-bottom:18px; padding:8px 22px; background:#60a5fa; color:#fff; border-radius:6px; text-decoration:none; font-size:1rem; font-weight:500; transition:all 0.2s; }',
         '.back-btn:hover { background:#3b82f6; transform:translateY(-2px); }',
         '.top-btn { position:fixed; right:36px; bottom:36px; z-index:99; background:#5bb8a7; color:#fff; border:none; border-radius:8px; padding:12px 22px; font-size:1.1rem; font-weight:600; cursor:pointer; box-shadow:0 2px 8px rgba(91, 184, 167, 0.3); transition:all 0.2s; }',
         '.top-btn:hover { background:#4a9485; transform:translateY(-2px); }',
        '.task-buttons { display: flex; flex-wrap: wrap; gap: 12px; margin-bottom: 30px; }',
        '.task-btn { padding: 10px 20px; background: #93c5fd; color: #1e3a8a; border: none; border-radius: 8px; cursor: pointer; font-size: 0.95rem; font-weight: 500; transition: all 0.2s; }',
        '.task-btn:hover { background: #60a5fa; color: #fff; transform: translateY(-2px); }',
        '.task-btn.active { background: #5bb8a7; color: #fff; box-shadow: 0 4px 12px rgba(91, 184, 167, 0.3); }',
        '.task-content { display: none; }',
        '.task-content.active { display: block; }',
        '.filter-btn { padding: 10px 20px; background: #93c5fd; color: #1e3a8a; border: none; border-radius: 8px; cursor: pointer; font-size: 0.95rem; font-weight: 500; margin-bottom: 20px; transition: all 0.2s; }',
        '.filter-btn:hover { background: #60a5fa; color: #fff; transform: translateY(-2px); }',
        '.filter-btn.active { background: #5bb8a7; color: #fff; box-shadow: 0 4px 12px rgba(91, 184, 167, 0.3); }',
        '.todo-caption { opacity: 0.8; color: #64748b; }',
        '.load-more-btn { display: block; margin: 20px auto; padding: 10px 20px; background: #60a5fa; color: #fff; border: none; border-radius: 8px; cursor: pointer; font-size: 0.95rem; font-weight: 500; transition: all 0.2s; }',
        '.load-more-btn:hover { background: #3b82f6; transform: translateY(-2px); }',
        '.video-counter { font-size: 0.9rem; color: #7f8c8d; margin-bottom: 10px; }',
        '.hidden-video { display: none; }',
        '.pagination { display:none; }',
        '</style>',
        '</head>',
        '<body>',
        '<a class="back-btn" href="../../index.html">&larr; Back</a>',
        '<br>'
        '<a class="back-btn" href="difference.html"">&rarr;Difference Page</a>'
        f'<h1>{os.path.basename(dataset_dir)} Dataset Viewer</h1>',
        '<button class="filter-btn" id="filterBtn" onclick="toggleFilter()">Show Whole Dataset</button>',
        '<button class="top-btn" onclick="window.scrollTo({top:0,behavior:\'smooth\'});">return to top</button>'
    ]

    # 添加任务按钮区域
    html.append('<div class="task-buttons">')
    tasks = list(annotations.items())
    for i, (task, vids) in enumerate(tasks):
        if not vids:  # 跳过没有标注的任务
            continue
        slug = task[:30].replace(' ', '_').replace('/', '_')
        task_dir = os.path.join(samples_dir, slug)
        if not os.path.isdir(task_dir):
            continue
        
        # 为每个任务创建按钮
        task_id = f"task_{i}"
        html.append(f'<button class="task-btn" onclick="showTask(\'{task_id}\')">{task[:50]}{"..." if len(task) > 50 else ""}</button>')
    html.append('</div>')

    # 添加任务内容区域
    max_videos_per_task = 100  # 每个任务最多显示的视频数
    
    for i, (task, vids) in enumerate(tasks):
        slug = task[:30].replace(' ', '_').replace('/', '_')
        task_dir = os.path.join(samples_dir, slug)
        if not os.path.isdir(task_dir):
            continue

        # 只处理有标注的视频
        if not vids:  
            continue

        task_id = f"task_{i}"
        # 第一个任务默认显示
        display_class = "task-content active" if i == 0 else "task-content"
        
        html.append(f'<div id="{task_id}" class="{display_class}">')
        html.append('<div class="task-block">')
        html.append(f'<div class="instruction-origin">ORIGIN: {task.replace("<", "&lt;").replace(">", "&gt;")}</div>')

        video_files = [f for f in os.listdir(task_dir) if f.endswith('.mp4')]
        
        # 按 ep_id 数字大小排序视频文件
        def get_ep_id(fname):
            try:
                return int(fname.split('_ep')[-1].replace('.mp4', ''))
            except ValueError:
                return 0
        
        video_files.sort(key=get_ep_id)
        
        # 计算实际有标注的视频数量
        valid_videos = []
        for fname in video_files:
            ep_id = fname.split('_ep')[-1].replace('.mp4', '')
            if ep_id in vids:
                valid_videos.append(fname)
        
        total_videos = len(valid_videos)
        html.append(f'<div class="video-counter">Total Videos: {total_videos} (Showing first {min(max_videos_per_task, total_videos)})</div>')
        
        html.append('<div class="video-list">')
        
        displayed_count = 0
        # 仅预渲染前100个，剩余用JS按需追加
        for fname in valid_videos[:max_videos_per_task]:
            video_path = f'samples/{slug}/{fname}'
            ep_id = fname.split('_ep')[-1].replace('.mp4', '')
            caption = vids.get(ep_id, None)
            
            if caption is not None:
                is_todo = caption.strip().upper() == "TODO CAPTION"
                todo_class = "todo-caption" if is_todo else ""
                
                html.append(f'<div class="video-item" data-todo="{str(is_todo).lower()}" data-task="{task_id}">')
                html.append(f'<video data-src="{video_path}" preload="none" controls width="100%" loop autoplay muted></video>')
                html.append(f'<div class="caption"><b>Video:</b> {fname}</div>')
                html.append(f'<div class="instruction-modified {todo_class}">MODIFY : {caption}</div>')
                html.append('</div>')
                displayed_count += 1
        
        html.append('</div>')  # .video-list
        
        # 如果视频数量超过限制，添加"Load All"按钮
        if total_videos > max_videos_per_task:
            # 将剩余视频的精简信息挂到 data-attrs 上（base64编码的JSON），供前端按批次追加
            rest_list = []
            for fname in valid_videos[max_videos_per_task:]:
                ep_id = fname.split('_ep')[-1].replace('.mp4', '')
                caption = vids.get(ep_id, None)
                if caption is None:
                    continue
                is_todo = (caption.strip().upper() == "TODO CAPTION")
                video_path = f'samples/{slug}/{fname}'
                rest_list.append({
                    'src': video_path,
                    'todo': str(is_todo).lower(),
                    'fname': fname,
                    'caption': caption,
                })
            rest_json = json.dumps(rest_list, ensure_ascii=False)
            rest_b64 = base64.b64encode(rest_json.encode('utf-8')).decode('ascii')
            html.append(f'<button class="load-more-btn" onclick="loadAllVideos(\'{task_id}\')" id="{task_id}_load" data-batch="30" data-rest-b64="{rest_b64}">Load All (show remaining {total_videos - max_videos_per_task})</button>')
        
        html.append('</div>')  # .task-block
        html.append('</div>')  # .task-content

    # 添加JavaScript代码
    html.extend([
        '<script>',
        'let showAllVideos = false;',
        '// 懒加载：用 IntersectionObserver 将 data-src 填到 video.src',
        'const io = new IntersectionObserver((entries) => {',
        '  entries.forEach(entry => {',
        '    if (entry.isIntersecting) {',
        '      const video = entry.target;',
        '      const dataSrc = video.getAttribute("data-src");',
        '      if (dataSrc && !video.src) { video.src = dataSrc; }',
        '      io.unobserve(video);',
        '    }',
        '  });',
        '}, { rootMargin: "200px" });',
        '',
        'function showTask(taskId) {',
        '    // 隐藏所有任务内容',
        '    const allContents = document.querySelectorAll(".task-content");',
        '    allContents.forEach(content => content.classList.remove("active"));',
        '    ',
        '    // 移除所有按钮的active状态',
        '    const allButtons = document.querySelectorAll(".task-btn");',
        '    allButtons.forEach(btn => btn.classList.remove("active"));',
        '    ',
        '    // 显示选中的任务内容',
        '    const selectedContent = document.getElementById(taskId);',
        '    if (selectedContent) {',
        '        selectedContent.classList.add("active");',
        '    }',
        '    ',
        '    // 设置选中按钮的active状态',
        '    const selectedButton = event.target;',
        '    if (selectedButton) {',
        '        selectedButton.classList.add("active");',
        '    }',
        '}',
        '',
        'function toggleFilter() {',
        '    showAllVideos = !showAllVideos;',
        '    const filterBtn = document.getElementById("filterBtn");',
        '    ',
        '    if (showAllVideos) {',
        '        filterBtn.textContent = "Hide TODO Captions";',
        '        filterBtn.classList.add("active");',
        '        // 显示所有视频',
        '        document.querySelectorAll(".video-item[data-todo=\'true\']").forEach(item => {',
        '            item.style.display = "block";',
        '        });',
        '    } else {',
        '        filterBtn.textContent = "Show Whole Dataset";',
        '        filterBtn.classList.remove("active");',
        '        // 隐藏TODO标注的视频',
        '        document.querySelectorAll(".video-item[data-todo=\'true\']").forEach(item => {',
        '            item.style.display = "none";',
        '        });',
        '    }',
        '}',
        '',
        'function loadAllVideos(taskId) {',
        '  const btn = document.getElementById(`${taskId}_load`);',
        '  if (!btn) return;',
        '  const batch = parseInt(btn.getAttribute("data-batch")) || 30;',
        '  let rest;',
        '  try {',
        '    const b64 = btn.getAttribute("data-rest-b64") || "";',
        '    const jsonStr = b64 ? atob(b64) : "[]";',
        '    rest = JSON.parse(jsonStr);',
        '  } catch (e) { rest = []; }',
        '  if (rest.length === 0) { btn.style.display = "none"; return; }',
        '  const list = document.querySelector(`#${taskId} .video-list`);',
        '  const toAppend = rest.splice(0, batch);',
        '  // 生成并追加 DOM',
        '  toAppend.forEach(item => {',
        '    const wrapper = document.createElement("div");',
        '    wrapper.className = "video-item";',
        '    wrapper.setAttribute("data-todo", item.todo);',
        '    wrapper.setAttribute("data-task", taskId);',
        '    wrapper.innerHTML = `',
        '      <video data-src="${item.src}" preload="none" controls width="100%" loop autoplay muted></video>',
        '      <div class="caption"><b>Video:</b> ${item.fname}</div>',
        '      <div class="instruction-modified ${item.todo === "true" ? "todo-caption" : ""}">MODIFY : ${item.caption}</div>',
        '    `;',
        '    list.appendChild(wrapper);',
        '    const v = wrapper.querySelector("video");',
        '    if (v) io.observe(v);',
        '  });',
        '  // 回写剩余数据（继续以base64保存，避免引号冲突）',
        '  btn.setAttribute("data-rest-b64", btoa(JSON.stringify(rest)));',
        '  if (rest.length === 0) { btn.style.display = "none"; }',
        '}',
        '',
        '// 初始隐藏TODO标注的视频',
        'document.addEventListener("DOMContentLoaded", function() {',
        '    // 初始状态：隐藏TODO标注的视频',
        '    document.querySelectorAll(".video-item[data-todo=\'true\']").forEach(item => { item.style.display = "none"; });',
        '    // 观察现有视频做懒加载',
        '    document.querySelectorAll("video[data-src]").forEach(v => io.observe(v));',
        '});',
        '</script>',
        '</body>',
        '</html>'
    ])

    with open(os.path.join(dataset_dir, 'index.html'), 'w', encoding='utf-8') as f:
        f.write('\n'.join(html))
    print(f"已生成: {os.path.join(dataset_dir, 'index.html')}")

# 用法示例
modifications_root = "/home2/qrchen/embodied-datasets/Modifications"

#data_list =  os.listdir(modifications_root)
data_list=[
        "droid",
        "dobbe",
        "fmb",
        "aloha_mobile",
        "io_ai_tech",
        "robo_set",
        "uiuc_d3field",
        "utaustin_mutex",
        "berkeley_fanuc_manipulation",
        "cmu_playing_with_food",
        "cmu_play_fusion",
        "cmu_stretch",
        "mimic_play",
        "bridge",
        "kuka"
]
for dataset_name in data_list:
    dataset_dir = os.path.join(modifications_root, dataset_name)
    if not os.path.isdir(dataset_dir):
        print(f"跳过 {dataset_dir}")
        continue
    try:
        generate_modifications_index(dataset_dir, videos_per_row=4)
    except Exception as e:
        print(f"跳过 {dataset_dir}，原因: {e}")

已生成: /home2/qrchen/embodied-datasets/Modifications/droid/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/dobbe/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/fmb/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/aloha_mobile/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/io_ai_tech/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/robo_set/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/uiuc_d3field/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/utaustin_mutex/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/berkeley_fanuc_manipulation/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/cmu_playing_with_food/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/cmu_play_fusion/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/cmu_stretch/index.html
已生成: /home2/qrchen/embodied-datasets/Modifications/mimic_play/index.html
已生成: /home2/qrchen/embodied-

## differencr pair.json

In [9]:
import os
import json
import base64


def generate_differences_html(dataset_dir='.', difference_file='difference.json', output_file='difference.html'):
    # 读取difference.json文件
    if not os.path.exists(difference_file):
        print(f"not find {difference_file}")
        return
    with open(difference_file, 'r', encoding='utf-8') as f:
        differences = json.load(f)
    
    # 获取所有样本目录
    samples_dir = os.path.join(dataset_dir, 'samples')
    
    # 开始构建HTML
    html = [
        '<!DOCTYPE html>',
        '<html lang="zh-cn">',
        '<head>',
        '<meta charset="UTF-8">',
        '<title>Trajectory Differences Visualization</title>',
        '<style>',
        "body { font-family: 'Segoe UI', Arial, sans-serif; background: #f7f7fa; padding: 30px; }",
        '.tasks-row { display: flex; flex-direction: column; gap: 48px; margin-bottom: 48px; }',
        '.task-block { background: #fff; border-radius: 14px; box-shadow: 0 2px 12px #e0e0f6; padding: 28px; width: 100%; }',
        '.task-title { font-size: 1.3rem; color: #3b82f6; font-weight: 600; margin-bottom: 20px; padding-bottom: 10px; border-bottom: 2px solid #bae6fd; }',
        '.difference-section { margin-bottom: 30px; }',
        '.difference-type { font-size: 1.1rem; color: #5bb8a7; font-weight: 500; margin: 25px 0 15px 0; padding-bottom: 8px; border-bottom: 1px dashed #93e1d8; }',
        '.difference-title { font-size: 1rem; color: #3b82f6; font-weight: 500; margin: 5px 0 10px 0; }',
        '.video-pair { display: flex; flex-wrap: wrap; gap: 20px; margin: 20px 0 30px 0; }',
        '.video-container { width: calc(25% - 15px); min-width: 250px; display: flex; flex-direction: column; }',
        '.video-item { border-radius: 8px; box-shadow: 0 1px 6px #eee; padding: 12px; flex: 1; display: flex; flex-direction: column; }',
        '.video-row { display: flex; flex-wrap: wrap; gap: 20px; margin-bottom: 20px; }',
        '.video-caption { font-size: 1.1rem; color: #7f8c8d; margin-top: 15px; }',
        'video { width: 100%; border-radius: 6px; background: #000; max-height: 400px; object-fit: contain; flex: 1; }',
        '.back-btn { display:inline-block; margin-bottom:18px; padding:8px 22px; background:#60a5fa; color:#fff; border-radius:6px; text-decoration:none; font-size:1rem; font-weight:500; transition:all 0.2s; }',
        '.back-btn:hover { background:#3b82f6; transform:translateY(-2px); }',
        '.top-btn { position:fixed; right:36px; bottom:36px; z-index:99; background:#5bb8a7; color:#fff; border:none; border-radius:8px; padding:12px 22px; font-size:1.1rem; font-weight:600; cursor:pointer; box-shadow:0 2px 8px rgba(91, 184, 167, 0.3); transition:all 0.2s; }',
        '.top-btn:hover { background:#4a9485; transform:translateY(-2px); }',
        '.task-buttons { display: flex; flex-wrap: wrap; gap: 12px; margin-bottom: 30px; }',
        '.task-btn { padding: 10px 20px; background: #93c5fd; color: #1e3a8a; border: none; border-radius: 8px; cursor: pointer; font-size: 0.95rem; font-weight: 500; transition: all 0.2s; }',
        '.task-btn:hover { background: #60a5fa; color: #fff; transform: translateY(-2px); }',
        '.task-btn.active { background: #5bb8a7; color: #fff; box-shadow: 0 4px 12px rgba(91, 184, 167, 0.3); }',
        '.task-content { display: none; }',
        '.task-content.active { display: block; }',
        '.highlight { background-color: #fffacd; padding: 2px 5px; border-radius: 3px; }',
        '.step-description { margin: 10px 0 20px 0; padding: 10px; background: #f0f7ff; border-radius: 6px; }',
        '.difference-block { margin-bottom: 30px; padding: 15px; background: #f0f9ff; border-radius: 10px; border: 1px solid #bae6fd; overflow: hidden; box-shadow: 0 2px 8px rgba(96, 165, 250, 0.1); }',
        'video { width: 100%; border-radius: 6px; background: #000; max-height: 400px; object-fit: contain; flex: 1; cursor: pointer; }',
        '.video-play-btn { position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); width: 50px; height: 50px; background: rgba(0,0,0,0.5); border-radius: 50%; display: flex; align-items: center; justify-content: center; color: white; font-size: 24px; pointer-events: none; opacity: 0; transition: opacity 0.2s; }',
        '.video-wrapper { position: relative; }',
        'video:hover + .video-play-btn, .video-play-btn:hover { opacity: 1; }',
        '</style>',
        '</head>',
        '<body>',
        '<a class="back-btn" href="index.html">&larr; Back</a>',
        '<h1>Trajectory Differences Visualization</h1>',
        '<button class="top-btn" onclick="window.scrollTo({top:0,behavior:\'smooth\'});">return to top</button>'
    ]

    # 添加任务按钮区域
    html.append('<div class="task-buttons">')
    tasks = list(differences.items())
    for i, (task, _) in enumerate(tasks):
        task_id = f"task_{i}"
        active_class = "active" if i == 0 else ""
        html.append(f'<button class="task-btn {active_class}" onclick="showTask(\'{task_id}\', this)">{task[:50]}{"..." if len(task) > 50 else ""}</button>')
    html.append('</div>')

    # 添加任务内容区域
    for i, (task, diff_types) in enumerate(tasks):
        task_id = f"task_{i}"
        display_class = "task-content active" if i == 0 else "task-content"
        
        html.append(f'<div id="{task_id}" class="{display_class}">')
        html.append('<div class="task-block">')
        html.append(f'<div class="task-title">Task: {task}</div>')
        
        for diff_type, steps in diff_types.items():
            html.append('<div class="difference-section">')
            html.append(f'<div class="difference-type">Difference: {diff_type}</div>')
            
            if "description" in steps:
                html.append(f'<div class="step-description">{steps["description"]}</div>')
            
            ep_pairs = [(k, v) for k, v in steps.items() if k.isdigit()]
            
            # 处理所有轨迹，每三个一组
            html.append('<div class="difference-block">')
            html.append('<div class="video-pair">')
            
            for j in range(0, len(ep_pairs)):
                
                # 获取当前轨迹
                ep_id, description = ep_pairs[j]
                slug = task[:30].replace(' ', '_').replace('/', '_')
                task_video_dir = os.path.join(samples_dir, slug)
                
                video_files = []
                if os.path.exists(task_video_dir):
                    video_files = [f for f in os.listdir(task_video_dir) 
                                    if f.endswith('.mp4') and f'_ep{ep_id}.mp4' in f]
                
                html.append('<div class="video-container">')
                html.append(f'<div class="difference-title">Episode {ep_id}:</div>')
                html.append('<div class="video-item">')
                    
                if video_files:
                    video_path = f'samples/{slug}/{video_files[0]}'
                    # 修改这里：移除controls属性，添加autoplay和muted
                    html.append('<div class="video-wrapper">')
                    html.append(f'<video src="{video_path}" preload="auto" width="100%" loop muted playsinline onclick="togglePlayPause(this)"></video>')
                    html.append('<div class="video-play-btn">▶</div>')
                    html.append('</div>')
                else:
                    html.append('<div style="background:#eee; padding:40px; text-align:center;">Video not found</div>')
                
                html.append(f'<div class="video-caption">{description}</div>')
                html.append('</div>')  # .video-item
                html.append('</div>')  # .video-container
                        
                                                # 每四个视频后添加新的行
                if (j + 1) % 4 == 0 and j < len(ep_pairs) - 1:
                    html.append('</div>')  # 结束当前 video-pair
                    html.append('<div class="video-pair">')  # 开始新的一行
                    
            # 在所有视频处理完后关闭最后的容器
            html.append('</div>')  # .video-pair
            html.append('</div>')  # .difference-block
            
            html.append('</div>')  # .difference-section
        
        html.append('</div>')  # .task-block
        html.append('</div>')  # .task-content

    # 添加JavaScript代码
    html.extend([
        '<script>',
        '// 显示选中的任务内容',
        'function showTask(taskId, clickedBtn) {',
        '    // 隐藏所有任务内容',
        '    document.querySelectorAll(".task-content").forEach(content => {',
        '        content.classList.remove("active");',
        '    });',
        '    ',
        '    // 移除所有按钮的active状态',
        '    document.querySelectorAll(".task-btn").forEach(btn => {',
        '        btn.classList.remove("active");',
        '    });',
        '    ',
        '    // 显示选中的任务内容',
        '    const selectedContent = document.getElementById(taskId);',
        '    if (selectedContent) {',
        '        selectedContent.classList.add("active");',
        '    }',
        '    ',
        '    // 设置选中按钮的active状态',
        '    if (clickedBtn) {',
        '        clickedBtn.classList.add("active");',
        '    }',
        '}',
        '',
        '// 切换视频的播放/暂停状态',
        'function togglePlayPause(video) {',
        '    if (video.paused) {',
        '        video.play();',
        '    } else {',
        '        video.pause();',
        '    }',
        '    // 更新播放按钮显示',
        '    const playBtn = video.nextElementSibling;',
        '    if (playBtn && playBtn.classList.contains("video-play-btn")) {',
        '        playBtn.textContent = video.paused ? "▶" : "❚❚";',
        '        playBtn.style.opacity = "1";',
        '        setTimeout(() => { playBtn.style.opacity = "0"; }, 1000);',
        '    }',
        '}',
        '',
        '// 自动播放所有视频',
        'function autoPlayVideos() {',
        '    document.querySelectorAll("video").forEach(video => {',
        '        // 确保视频已加载',
        '        if (video.readyState === 0) {',
        '            video.load();',
        '        }',
        '        ',
        '        // 尝试自动播放',
        '        const playPromise = video.play();',
        '        ',
        '        // 处理自动播放可能被阻止的情况',
        '        if (playPromise !== undefined) {',
        '            playPromise.catch(error => {',
        '                console.log("Auto-play prevented, showing controls for video:", video);',
        '                video.controls = true;',
        '            });',
        '        }',
        '    });',
        '}',
        '',
        '// 页面加载完成后初始化',
        'document.addEventListener("DOMContentLoaded", function() {',
        '    autoPlayVideos();',
        '    ',
        '    // 为视频添加鼠标悬停事件',
        '    document.querySelectorAll(".video-wrapper").forEach(wrapper => {',
        '        const video = wrapper.querySelector("video");',
        '        const playBtn = wrapper.querySelector(".video-play-btn");',
        '        ',
        '        wrapper.addEventListener("mouseenter", () => {',
        '            if (playBtn) playBtn.style.opacity = "1";',
        '        });',
        '        ',
        '        wrapper.addEventListener("mouseleave", () => {',
        '            if (playBtn) playBtn.style.opacity = "0";',
        '        });',
        '    });',
        '});',
        '</script>',
        '</body>',
        '</html>'
    ])

    # 写入输出文件
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(html))
    print(f"已生成: {output_file}")


modifications_root = "/home2/qrchen/embodied-datasets/Modifications"



for dataset_name in data_list:
    dataset_dir = os.path.join(modifications_root, dataset_name)
    if not os.path.isdir(dataset_dir):
        continue
    # try:
    generate_differences_html(dataset_dir, os.path.join(dataset_dir, 'difference.json'), os.path.join(dataset_dir, 'difference.html'))
    # except Exception as e:
    #     print(f"跳过 {dataset_dir}，原因: {e}")

已生成: /home2/qrchen/embodied-datasets/Modifications/droid/difference.html
已生成: /home2/qrchen/embodied-datasets/Modifications/dobbe/difference.html
已生成: /home2/qrchen/embodied-datasets/Modifications/fmb/difference.html
not find /home2/qrchen/embodied-datasets/Modifications/aloha_mobile/difference.json
已生成: /home2/qrchen/embodied-datasets/Modifications/io_ai_tech/difference.html
已生成: /home2/qrchen/embodied-datasets/Modifications/robo_set/difference.html
已生成: /home2/qrchen/embodied-datasets/Modifications/uiuc_d3field/difference.html
已生成: /home2/qrchen/embodied-datasets/Modifications/utaustin_mutex/difference.html
已生成: /home2/qrchen/embodied-datasets/Modifications/berkeley_fanuc_manipulation/difference.html
not find /home2/qrchen/embodied-datasets/Modifications/cmu_playing_with_food/difference.json
已生成: /home2/qrchen/embodied-datasets/Modifications/cmu_play_fusion/difference.html
not find /home2/qrchen/embodied-datasets/Modifications/cmu_stretch/difference.json
已生成: /home2/qrchen/embodied-d

## 生成单独的 difference html 文件

In [ ]:
import os
import json

def generate_unified_visualization(modifications_root, output_file='unified_visualization.html', videos_per_row=4):
    # 读取两种不同的difference文件
    all_differences = {}  # 用于存储同一数据集的differences
    data_list = [
        "dobbe", "fmb", "aloha_mobile", "droid", "io_ai_tech",
        "robo_set", "uiuc_d3field", "utaustin_mutex",
        "berkeley_fanuc_manipulation", "cmu_playing_with_food",
        "cmu_play_fusion", "cmu_stretch","mimic_play","kuka","bc_z"
    ]
    
    # 读取同一数据集的differences
    for dataset_name in data_list:
        dataset_dir = os.path.join(modifications_root, dataset_name)
        if not os.path.isdir(dataset_dir):
            continue
            
        difference_file = os.path.join(dataset_dir, 'difference.json')
        if not os.path.exists(difference_file):
            continue
            
        try:
            with open(difference_file, 'r', encoding='utf-8') as f:
                differences = json.load(f)
                all_differences[dataset_name] = differences
        except Exception as e:
            print(f"跳过 {dataset_name} 的difference.json，原因: {e}")
    
    # 读取跨数据集的differences
    with open('/home2/qrchen/embodied-datasets/scripts/xintong/difference.json', 'r', encoding='utf-8') as f:
        cross_dataset_differences = json.load(f)
    
    # 计算总的difference数量
    total_differences_same_dataset = sum(
        sum(1 for diff_type in task_diffs.keys() if diff_type != "description")
        for dataset_diffs in all_differences.values()
        for task_diffs in dataset_diffs.values()
    )
    total_differences_cross_dataset = len(cross_dataset_differences)
    total_differences = total_differences_same_dataset + total_differences_cross_dataset
    
    # 开始构建HTML
    html = [
        '<!DOCTYPE html>',
        '<html lang="zh-cn">',
        '<head>',
        '<meta charset="UTF-8">',
        '<title>Unified Trajectory Visualization</title>',
        '<style>',
        "body { font-family: 'Segoe UI', Arial, sans-serif; background: #f7f7fa; padding: 30px; }",
        '.tasks-row { display: flex; flex-direction: column; gap: 48px; margin-bottom: 48px; }',
        '.task-block { background: #fff; border-radius: 14px; box-shadow: 0 2px 12px #e0e0f6; padding: 28px; width: 100%; }',
        '.task-title { font-size: 1.3rem; color: #3b82f6; font-weight: 600; margin-bottom: 20px; padding-bottom: 10px; border-bottom: 2px solid #bae6fd; }',
        '.difference-section { margin-bottom: 30px; }',
        '.difference-type { font-size: 1.1rem; color: #5bb8a7; font-weight: 500; margin: 25px 0 15px 0; padding-bottom: 8px; border-bottom: 1px dashed #93e1d8; }',
        '.difference-title { font-size: 1rem; color: #3b82f6; font-weight: 500; margin: 5px 0 10px 0; }',
        '.video-pair { display: flex; flex-wrap: wrap; gap: 20px; margin: 20px 0 30px 0; }',
        '.video-container { width: calc(25% - 15px); min-width: 250px; display: flex; flex-direction: column; }',
        '.video-item { border-radius: 8px; box-shadow: 0 2px 8px rgba(96, 165, 250, 0.1); padding: 12px; background: #f0f9ff; border: 1px solid #bae6fd; }',
        '.video-caption { font-size: 1.1rem; color: #7f8c8d; margin-top: 15px; }',
        '.dataset-label { font-size: 0.9rem; color: #94a3b8; margin-top: 8px; text-align: right; font-style: italic; }',
        '.back-btn { display:inline-block; margin-bottom:18px; padding:8px 22px; background:#60a5fa; color:#fff; border-radius:6px; text-decoration:none; font-size:1rem; font-weight:500; transition:all 0.2s; }',
        '.back-btn:hover { background:#3b82f6; transform:translateY(-2px); }',
        '.top-btn { position:fixed; right:36px; bottom:36px; z-index:99; background:#5bb8a7; color:#fff; border:none; border-radius:8px; padding:12px 22px; font-size:1.1rem; font-weight:600; cursor:pointer; box-shadow:0 2px 8px rgba(91, 184, 167, 0.3); transition:all 0.2s; }',
        '.top-btn:hover { background:#4a9485; transform:translateY(-2px); }',
        '.task-header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 20px; }',
        '.dataset-name { font-size: 1rem; color: #94a3b8; font-style: italic; }',
        '.section-title { font-size: 2rem; color: #1e40af; margin: 40px 0 20px 0; padding-bottom: 15px; border-bottom: 3px solid #3b82f6; }',
        '</style>',
        '</head>',
        '<body>',
        '<a class="back-btn" href="index.html">&larr; Back</a>',
        f'<h1>Unified Trajectory Visualization (Total: {total_differences} differences)</h1>',
        '<button class="top-btn" onclick="window.scrollTo({top:0,behavior:\'smooth\'});">return to top</button>'
    ]
    
    # 计数器
    difference_counter = 1
    
    # 第一部分：显示同一数据集中的differences
    html.append('<div class="section-title">Part 1: Differences within Datasets</div>')
    
    for dataset_name, differences in all_differences.items():
        dataset_dir = os.path.join(modifications_root, dataset_name)
        samples_dir = os.path.join(dataset_dir, 'samples')
        
        for task, diff_types in differences.items():
            html.append('<div class="task-block">')
            html.append('<div class="task-header">')
            html.append(f'<div class="task-title">Task: {task}</div>')
            html.append(f'<div class="dataset-name">Dataset: {dataset_name}</div>')
            html.append('</div>')
            
            for diff_type, steps in diff_types.items():
                if diff_type == "description":
                    continue
                html.append('<div class="difference-section">')
                html.append(f'<div class="difference-type">{difference_counter}. Difference: {diff_type}</div>')
                difference_counter += 1
                
                if "description" in steps:
                    html.append(f'<div class="step-description">{steps["description"]}</div>')
                
                ep_pairs = [(k, v) for k, v in steps.items() if k.isdigit()]
                
                html.append('<div class="difference-block">')
                html.append('<div class="video-pair">')
                
                for j in range(0, len(ep_pairs)):
                    ep_id, description = ep_pairs[j]
                    slug = task[:30].replace(' ', '_').replace('/', '_')
                    task_video_dir = os.path.join(samples_dir, slug)
                    
                    video_files = []
                    if os.path.exists(task_video_dir):
                        video_files = [f for f in os.listdir(task_video_dir) 
                                     if f.endswith('.mp4') and f'_ep{ep_id}.mp4' in f]
                    
                    html.append('<div class="video-container">')
                    html.append(f'<div class="difference-title">Episode {ep_id}:</div>')
                    html.append('<div class="video-item">')
                    
                    if video_files:
                        video_path = f'Modifications/{dataset_name}/samples/{slug}/{video_files[0]}'
                        html.append('<div class="video-wrapper">')
                        html.append(f'<video src="{video_path}" controls preload="auto" width="100%" loop autoplay muted playsinline></video>')
                        html.append('</div>')
                    else:
                        html.append('<div style="background:#eee; padding:40px; text-align:center;">Video not found</div>')
                    
                    html.append(f'<div class="video-caption">{description}</div>')
                    html.append(f'<div class="dataset-label">Dataset: {dataset_name}</div>')
                    html.append('</div>')  # .video-item
                    html.append('</div>')  # .video-container
                    
                    if (j + 1) % videos_per_row == 0 and j < len(ep_pairs) - 1:
                        html.append('</div>')  # 结束当前 video-pair
                        html.append('<div class="video-pair">')  # 开始新的一行
                
                html.append('</div>')  # .video-pair
                html.append('</div>')  # .difference-block
                html.append('</div>')  # .difference-section
            
            html.append('</div>')  # .task-block
    
    # 第二部分：显示跨数据集的differences
    html.append('<div class="section-title">Part 2: Cross-Dataset Differences</div>')
    
    for difference_name, datasets in cross_dataset_differences.items():
        html.append('<div class="task-block">')
        html.append('<div class="task-header">')
        html.append(f'<div class="task-title">{difference_counter}. {difference_name}</div>')
        html.append('</div>')
        
        html.append('<div class="video-pair">')
        
        for dataset_name, tasks in datasets.items():
            for task_name, episodes in tasks.items():
                slug = task_name[:30].replace(' ', '_').replace('/', '_')
                for ep_id, caption in episodes.items():
                    # 尝试两种可能的视频文件名格式
                    video_name1 = f"{slug}_ep{ep_id}.mp4"
                    video_name2 = f"_ep{ep_id}.mp4"
                    
                    video_path1 = os.path.join(modifications_root, dataset_name, "samples", slug, video_name1)
                    video_path2 = os.path.join(modifications_root, dataset_name, "samples", slug, video_name2)
                
                    html.append('<div class="video-container">')
                    html.append('<div class="video-item">')
                    
                    if os.path.exists(video_path1):
                        relative_video_path = f"Modifications/{dataset_name}/samples/{slug}/{video_name1}"
                        html.append('<div class="video-wrapper">')
                        html.append(f'<video src="{relative_video_path}" controls preload="auto" width="100%" loop autoplay muted playsinline></video>')
                        html.append('</div>')
                    elif os.path.exists(video_path2):
                        relative_video_path = f"Modifications/{dataset_name}/samples/{slug}/{video_name2}"
                        html.append('<div class="video-wrapper">')
                        html.append(f'<video src="{relative_video_path}" controls preload="auto" width="100%" loop autoplay muted playsinline></video>')
                        html.append('</div>')
                    else:
                        html.append('<div style="background:#eee; padding:40px; text-align:center;">Video not found</div>')
                        print(f"No video found at either path")
                        # 检查目录是否存在
                        dir_path = os.path.join(modifications_root, dataset_name, "samples", slug)
                        if os.path.exists(dir_path):
                            print(f"Directory exists: {dir_path}")
                        else:
                            print(f"Directory does not exist: {dir_path}")
                    
                    html.append(f'<div class="video-caption">{caption}</div>')
                    html.append(f'<div class="dataset-label">{dataset_name}/{task_name}</div>')
                    html.append('</div>')  # .video-item
                    html.append('</div>')  # .video-container
        
        html.append('</div>')  # .video-pair
        html.append('</div>')  # .task-block
        difference_counter += 1
    
    # 添加JavaScript代码
    html.extend([
        '<script>',
        'document.addEventListener("DOMContentLoaded", function() {',
        '    document.querySelectorAll("video").forEach(video => {',
        '        video.load();',
        '        const playPromise = video.play();',
        '        if (playPromise !== undefined) {',
        '            playPromise.catch(error => {',
        '                console.log("Auto-play prevented:", error);',
        '            });',
        '        }',
        '        video.addEventListener("ended", function() {',
        '            this.currentTime = 0;',
        '            this.play();',
        '        });',
        '    });',
        '});',
        '</script>',
        '</body>',
        '</html>'
    ])
    
    # 写入输出文件
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(html))
    print(f"已生成: {output_file}")

if __name__ == "__main__":
    modifications_root = "/home2/qrchen/embodied-datasets/Modifications"
    output_file = "/home2/qrchen/embodied-datasets/unified_visualization.html"
    generate_unified_visualization(modifications_root, output_file)

已生成: /home2/qrchen/embodied-datasets/difference.html
